# Coding Outreach Group Summer Workshop
# BIDS: HeuDiConv + fMRIPrep

07/29/2021

__**Content creator:**__ [Elizabeth (Liz) Beard](https://github.com/elizabethbeard)

**WARNING: THIS WORKSHOP IS UNDER CONSTRUCTION, CHANGES WILL BE MADE LATER**

## Set up & Prerequisites
For this workshop, it's critical that participants feel comfortable on the command line in bash, as well as using logic operatores such as `if`, `and`, `or`. Be sure to check out the [README]() for the workshop to make sure you've completed the following steps:
1. Installed docker onto your local computer
2. Pulled the HeuDiConv:latest and fMRIPrep:stable containers
3. Downloaded and saved your FreeSurfer license.txt onto your local computer

## Description
This workshop introduces attendees to the concept of using Brain Imaging Data Structures (BIDS) and two crucial BIDS Apps for neuroimaging researchers: HeuDiConv and fMRIPrep using Docker.

## Outline

| Topic | Description | Time |
| --- | --- | --- |
| Intro | What is BIDS? Why is it important? | 5 min |
| Docker | What is Docker? | 5 min |
| Tutorial 1 | HeuDiConv example | 25 min |
| Tutorial 2 | fMRIPrep walkthrough | 15 min |
| Caveats & Troubleshooting | resources for troubleshooting | 5 min |

# Intro

In [2]:
## ADD INTRO VIDEO HERE ##

# Docker - What is it?
Docker is a system to build and share *containers* or standardized units of software. 

Okay... but what is a *container*? I'm really not a software developer, so let's refer to the [Docker website](https://www.docker.com/resources/what-container):
> A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another.

Essentially, I like to think of docker containers as somewhat analogous to Conda Environments but specifically for software. It ultimately allows developers to not have to worry as much about users working on different platforms than the one they develop on. 

This video provides a good overview of why containerized software is a good idea (thanks to Camille Johnston for sharing!):

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/zfclTRtKOSM?start=2" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

# 1. HeuDiConv
Developed by the team at nipype, [HeuDiConv](https://heudiconv.readthedocs.io/en/latest/) is a "flexible DICOM converter for organizing brain imaging data into structured directory layouts". In this tutorial, we'll be going through the entire HeuDiConv workflow and converting a single subject's MRI data into BIDS format.

** ADD roadmap.png HERE **

## Creating the .heudiconv directory
You should have already pulled the HeuDiConv container from docker using `docker pull nipy/heudiconv:latest` when setting up for the workshop, but if you haven't be sure to do that now.

To begin, we'll run a 'blank' version of the HeuDiConv command in order to generate the heuristic file template (more on that in a moment) in the terminal. To get a better understanding of what each of the flags in this command reference, be sure to check out [HeuDiConv's documentation](https://heudiconv.readthedocs.io/en/latest/usage.html).

``` bash
docker run --rm -it \
-v /path/to/your/raw/imaging/data/:/base \
nipy/heudiconv:latest \
-d /base/raw-{subject}-level-file-path/*/DICOM/*.dcm \
-o /base/bids/ \
-f convertall \
-s 004 \
-c none
```

Let's take a look at what each of these lines of code does more specifically:
- `docker run --rm -it \` is calling Docker, and telling it we want to run something.
- `-v /path/to/your/raw/imaging/data/:/base \` *mounts* our base directory to the docker image so that the software knows where to look for the files we want to use.
- `nipy/heudiconv:latest` tells Docker which container we want to run
- `-d /base/raw-{subject}-level-file-path/*/DICOM/*.dcm` is the pattern that ALL of the files we want to convert will follow. It is CRITICAL that this matches the file path of your subjects imaging data correctly.
    - The `{subject}` key will always be replaced by the specificed subject number below.
    - The `*` is known as a 'wild card' and will search through any files at that path level. So this search will go into every DICOM folder and look for files that end in .dcm.
- `-o /base/bids/` specifies the output directory. You should already have the bids directory in your workshop folder, but if you don't you need to make it before you run the command!
- `-f convertall` tells HeuDiConv which heuristic file to use. Right now, we want it to generate a heuristic template for us which is what this command does.
- `s ###` is the subject number we want to convert.
- `c none` is the converter we want to use. Since this is the dry run, we don't want to convert anything just yet.

**NOTE**: Before running your command, I recommend writing out the entire thing in a .txt file that you can copy and paste into the terimal after triple checking that your file paths are correct. More often that not, HeuDiConv doesn't work because there is an error in the way the paths are specified.

Once you've run the command successfully, you should see output that looks like this:
```bash
INFO: Running heudiconv version 0.9.0 latest 0.9.0
INFO: Need to process 1 study sessions
INFO: PROCESSING STARTS: {'subject': '004', 'outdir': '/base/bids/', 'session': None}
INFO: Processing 1547 dicoms
INFO: Analyzing 1547 dicoms
INFO: Generated sequence info for 10 studies with 1538 entries total
INFO: PROCESSING DONE: {'subject': '004', 'outdir': '/base/bids/', 'session': None}
```
Now let's go into our output directory (`bids/`) and look for that .heudiconv file.

In [5]:
%%bash
ls -a bids/

.
..
.heudiconv


Notice how there is the invisible `.heudiconv` file? Let's take a peak and see what's in there. We should see a subject level folder. Let's look at the info folder inside that too.

In [6]:
%%bash
ls -a bids/.heudiconv/

.
..
004


In [8]:
%%bash
ls -a bids/.heudiconv/004/info

.
..
004.auto.txt
004.edit.txt
dicominfo.tsv
filegroup.json
heuristic.py


There are two critical items that we generated as a result of this dry run:
1. `heuristic.py`: the heuristics file which will let heudiconv accurately comb through our files to better convert them into BIDS format
2. `dicominfo.tsv`: a tsv file that contains the necessary information we'll use to develop our heuristics file

## Setting up your heuristics file
Next, we'll go ahead and open the heuristic.py script to add in the heuristics we want to use.
- ***note***: Since this folder starts with a `.`, it will likely be invisible on your Finder window. To display invisible folders press `Shift + Command + .` in the Finder.

We'll preimiarly make edits in the `def infotodict` section of the code.

`data` is an example for how to create a key. Keys define the type of scan you want your heuristic file to look for and will format your filenames according BIDS specifications. (For more info about BIDS naming conventions, look [here](https://bids-specification.readthedocs.io/en/stable/)). So for our purposes, we may include objects such as:

``` python
T1w = create_key('sub-{subject}/anat/sub-{subject}_T1w')
rest = create_key('sub-{subject}/func/sub-{subject}_task-rest_rec-{rec}_run-{item:01d}_bold')
```

This is where it helps to be at least somewhat familiar with python constructs but essentially, you are using logic statements that will parse through the `dicominfo.tsv` file and re-name your files in the proper format (that YOU specify). One of the benefits of using HeuDiConv is that you can update 

After you've updated your `heuristic.py` script, save it with the rest of your code (it's recommended to have a `project/bids/code` folder) since you'll reference it later.

For the sake of time today, I've gone ahead and created a nearly completed `heuristic.py` file in our `bids/code` folder for the example subject today -- but we're missing a task that we'd like to convert.

**TASK**: Add the `CanonicalTask` to the heuristic.py script by referencing the `dicominfo.tsv` file.

## Let's do this.
OKAY, so after you've gone through *all* this setup, we should finally be ready to re-run the heudiconv app to convert our files to .nii format and shift them into BIDS format.

To do this, we'll have to change some of the initial code from generating the heuristic file:

``` bash
docker run --rm -it \
-v /path/to/your/raw/imaging/data/:/base \
nipy/heudiconv:latest \
-d /base/raw-{subject}-level-file-path/*/DICOM/*.dcm \
-o /base/bids/ \
-f /base/bids/code/heuristic.py \
-s 004 \
-c dcm2niix \
-b
```

***Note:*** There are a number of different converters that work with [BIDS apps](http://bids.neuroimaging.io/). Here I'm using `dcm2niix` since we're also (likely) looking to convert our dicoms to .nii files. It's also the converter that works specifically with heudiconv.

Once you've run the command successfully, you'll see a bunch of output as the conversion script runs. Once a subject is finished, you should see output that looks like this:
```bash
INFO: PROCESSING DONE: {'subject': '004', 'outdir': '/base/bids/', 'session': None}
```
Now let's go into our output directory (`bids/`) and see if our data was converted into BIDS format, and our dicoms were converted to NIFTI format.

In [1]:
%%bash
ls -a bids/

.
..
.DS_Store
.heudiconv
CHANGES
README
code
dataset_description.json
sub-004
task-DMTask_bold.json


In [4]:
%%bash
ls -a bids/sub-004/*

bids/sub-004/sub-004_scans.json
bids/sub-004/sub-004_scans.tsv

bids/sub-004/anat:
.
..
sub-004_T1w.json
sub-004_T1w.nii.gz

bids/sub-004/fmap:
.
..
sub-004_magnitude1.json
sub-004_magnitude1.nii.gz
sub-004_magnitude2.json
sub-004_magnitude2.nii.gz
sub-004_phasediff.json
sub-004_phasediff.nii.gz

bids/sub-004/func:
.
..
sub-004_task-DMTask_run-1_bold.json
sub-004_task-DMTask_run-1_bold.nii.gz
sub-004_task-DMTask_run-1_events.tsv
sub-004_task-DMTask_run-2_bold.json
sub-004_task-DMTask_run-2_bold.nii.gz
sub-004_task-DMTask_run-2_events.tsv
sub-004_task-DMTask_run-3_bold.json
sub-004_task-DMTask_run-3_bold.nii.gz
sub-004_task-DMTask_run-3_events.tsv
sub-004_task-DMTask_run-4_bold.json
sub-004_task-DMTask_run-4_bold.nii.gz
sub-004_task-DMTask_run-4_events.tsv


## BONUS: Confirming BIDS format
There are a number of BIDS validation tools that you can use to confirm your data is indeed in BIDS format:
* [this program](https://github.com/bids-standard/bids-validator) runs locally and instructions are included on their webpage, it's probably better for checking large batches of data
* [this program](http://bids-standard.github.io/bids-validator/) allows you to validate a directory/file structure online

Let's use the online tool to see if our conversion worked! At first pass, it looks like heudiconv did at least *something*. There are some warnings that we can ignore for using fmriprep, since it doesn't use behavioral data. If you were to post your data online though, this would be a useful resource to make sure your data is in the most accurate BIDS format.

All in all, looks pretty good!

# 2. fMRIPrep
[fMRIPrep](https://fmriprep.org/en/stable/) is a "a functional magnetic resonance imaging (fMRI) data preprocessing pipeline that is designed to provide an easily accessible, state-of-the-art interface that is robust to variations in scan acquisition protocols and that requires minimal user input, while providing easily interpretable and comprehensive error and output reporting" developed by the Poldrack Lab.

In this tutorial, we'll review the steps for running fMRIPrep on your own data and machine. The conversion takes a long time so we won't actually be converting the data today; you're welcome to try it on the sample subject provided here on your own time.

## Execution

There are two different routes to using fMRIPrep. In this tutorial, we've installed a containerized version using Docker and will be executing fMRIPrep directly from the commandline. The developers of fMRIPrep also offer a less flexible, Python wrapper to run fMRIPrep which you can read more about [here](https://fmriprep.org/en/stable/usage.html?highlight=python%20wrapper#the-command-line-interface-of-the-docker-wrapper).

To preprocess your imaging dataset using fMRIPrep, your dataset must be in valid BIDS format, and it must include at least one T1w structural image and a BOLD series. Here is an example of what your fMRIPrep command might look like:

To get a better understanding of what each of the flags in this command reference, be sure to check out [fMRIPrep's documentation](https://fmriprep.org/en/stable/usage.html#command-line-arguments).

``` bash

docker run --rm -it \
-v /path/to/your/bids/data/:/base \
-v /path/to/your/output/file/derivatives:/out \
-v /path/to/your/freesurfer/liscense.txt:/opt/freesurfer/license.txt \
-v /path/to/where/you/want/to/store/temporary/files:/work \
nipreps/fmriprep:stable \
/data /out \
participant \
--skip-bids-validation \
--fs-no-reconall -w /work

```

Let's take a look at what each of these lines of code does more specifically:
- `docker run --rm -it \` is calling Docker, and telling it we want to run something.
- `-v /path/to/your/bids/data/:/base \` *mounts* our bids directory to the docker image so that the software knows where to look for the files we want to use.
- `-v /path/to/your/output/file/derivatives:/out \` *mounts* our output directory to the docker image so that the software knows where to save out files.
- `-v /path/to/your/freesurfer/liscense.txt:/opt/freesurfer/license.txt \` *mounts* our freesurfer license. fMRIPrep won't run if you don't have this option.
- `-v /path/to/where/you/want/to/store/temporary/files:/work \` *mounts* our working directory. This is an optional flag that allows you to use a different storage location for the temporary files fMRIPrep creates.
- `/data /out \` tells fMRIPrep where the data lives and where to save the data. If we had just bounted a `base` directory, we could further specify which folders we want to save out the data to.
- `participant \` specifies the analysis level we want to run, this is the only option available for fMRIPrep
    - **note**: If you don't specify a `--participant_label`, fMRIPrep will run on all of the subjects in a given bids directory.
- `--skip-bids-validation \` because we have those minor issues with some of the documentation files (that fMRIPrep doesn't use), we're going to skip the internal bids validation (it'll still raise the same errors we got using the BIDS validator).
- `--fs-no-reconall -w /work` The `--fs-no-reconall` flag is used to skip the freesurfer surface reconstructio. This saves a *lot* of time in your preprocessing. The `-w /work` flag tells fMRIPrep where to place and use temporary files.

After you've run the fMRIPrep command, there are three main outputs that will be saved in a BIDS Derivatives specification:
1. Visual QA (quality assessment) report
2. Derivatives (preprocessed data
3. Confounds (motion and other nuisance regressors)


## Additional Arguments
The fMRIPrep command has a number of different optional arguments to customize your preprocessing pipeline and fit your prepropressing needs. The best way to learn about them is to check out [fMRIPrep's documentation](https://fmriprep.org/en/stable/usage.html#command-line-arguments), but here's a short list of options you might want to consider:
- `--nprocs, --nthreads, etc` specify the maximum number of threads or processors to use. This is especially helpful if you're running processing on a computing cluster.
- `--verbose` increases the amount of information you get during each step of the preprocessing pipeline. Particularly helpful debugging your preprocessing.
- `--output-spaces` specifies different spaces to resample anotomical and functional images. More details [here](https://fmriprep.org/en/20.2.3/spaces.html).

# Troubleshooting and Other Resources
Possibly the hardest challenge in using HeuDiConv and fMRIPrep is getting started and troubleshooting. The error output in both programs can be challenging to parse. That being said, there are a number of different resources to try and troubleshoot your code:
- THE DOCUMENTATION. I cannot stress enough that directly referring to the documentation will help catch any stray typos or commands.
    + [HeuDiConv Documentation](https://heudiconv.readthedocs.io/en/latest/)
    + [fMRIPrep Documentation](https://fmriprep.org/en/stable/)
- [NeuroStars](https://neurostars.org/) is a forum for neuroscience researchers. Search for the HeuDiConv and fMRIPrep tags to see if others have had the same issues as you.
- GitHub. Both HeuDiConv and fMRIPrep are distributed via GitHub. If you think the issue you are running into from a software issue, be sure to check the issues section.
    + [HeuDiConv GitHub](https://github.com/nipy/heudiconv)
    + [fMRIPrep GitHub](https://github.com/nipreps/fmriprep)

## Other Tutorials
There are many other tutorials for both HeuDiConv and fMRIPrep. But, since both of these tools are in development, the tutorials may not have the most up-to-date version of the software.
- [HeuDiConv Tutorials](https://heudiconv.readthedocs.io/en/latest/tutorials.html)
- [Stanford Center for Reproducible Science fMRIPrep tutorial](https://reproducibility.stanford.edu/fmriprep-tutorial-running-the-docker-image/)
- [Andy's Brain Book fMRIPrep demo](https://andysbrainbook.readthedocs.io/en/latest/OpenScience/OS/fMRIPrep_Demo_1_Download.html)
- [Gelana Tostaeva's fMRIPrep tutorial](https://medium.com/@gelana/using-fmriprep-for-fmri-data-preprocessing-90ce4a9b85bd)

# Outro

In [5]:
## ADD OUTRO VIDEO HERE ## 